In [ ]:
##################################################################################################
##  This code runs a full CHSH Bell test between two Cyclical Quantum Memories (CQMs)
##  including setting polarizer angles, time-to-digital converter settings, storage lengths, 
##  initializing aquisition, and saving data. You can choose to recieve SMS notifications as well
##  
##  I have labels places in the code you will need to update, with the comment: "##update##"
##  
##  Good luck
##  
##  Created by Carson Evans, 2024
##  Last Updated:
##  March 13th, 2025 - Carson
##  
##################################################################################################

In [2]:
import serial
import time
import numpy as np
from typing import Any, Dict, Iterable, List
import re
import pyvisa
from datetime import datetime, timedelta
from pathlib import Path
import pickle

In [3]:
lab_data_path = 'C:/Users/username/Documents/Research/Lab_Data/'   ##update##

# Add an optional description of the data folder at the end of the folder name
directory_description = "_dualCQMs_spdc_photons"
# directory_description = "_dualCQMs_testing_PDC_polarizers"

# Get the current date as a string
current_date = datetime.now().strftime("%Y_%m%b_%d")

# Define the directory where you want to save the files and create it if it doesn't exist
output_dir = Path(lab_data_path + current_date + directory_description)
output_dir.mkdir(parents=True, exist_ok=True)

print(output_dir)

C:\Users\cevans\OneDrive\Documents\PhD_Research\Lab_Data\2024_10Oct_22_dualCQMs_spdc_photons


In [26]:
### Run this whole thing to initialize the majority of the functions ###

delay1 = 322500
delay2 = 320000
hist_center = 100000 # histograms centered at 100 ns
cycle_time = 27000
window_delay4 = 112000+24400 # Beginning of first window for 2 round trips
window_delay3 = window_delay4
window_delay2 = window_delay4 + hist_center
window_delay1 = window_delay2

cyc2_delays = {1: window_delay1,
               2: window_delay2,
               3: window_delay3,
               4: window_delay4}

def configure_tc_CQM1_triggers():
    try:
        tc = connect(DEFAULT_TC_ADDRESS)
    
        dlt = dlt_connect(output_dir, DEFAULT_DLT_PATH)
        
        zmq_exec(tc, f"DELA1:VALUe {delay1-2*hist_center}")
        zmq_exec(tc, f"DELA2:VALUe {delay2-2*hist_center}")
    
        # histogram1: input1 as start (tsco5) and unfiltered input2 as stop (tsco6)
        zmq_exec(tc, f"HIST1:REF:LINK {tsco['det1']};:HIST1:STOP:LINK {tsco['det3']}")
        zmq_exec(tc, f"HIST2:REF:LINK {tsco['det2']};:HIST2:STOP:LINK {tsco['det4']}")
        zmq_exec(tc, f"HIST3:REF:LINK {tsco['det2']};:HIST3:STOP:LINK {tsco['det3']}")
        zmq_exec(tc, f"HIST4:REF:LINK {tsco['det1']};:HIST4:STOP:LINK {tsco['det4']}")
    
    except (ConnectionError, DataLinkTargetError, NotADirectoryError) as e:
        text_my_phone("Configuration Error", str(e))
        print(e)

def configure_tc_CQM2_triggers():
    try:
        tc = connect(DEFAULT_TC_ADDRESS)
    
        dlt = dlt_connect(output_dir, DEFAULT_DLT_PATH)
        
        zmq_exec(tc, f"DELA1:VALUe {delay1}")
        zmq_exec(tc, f"DELA2:VALUe {delay2}")
    
        # histogram1: input1 as start (tsco5) and unfiltered input2 as stop (tsco6)
        zmq_exec(tc, f"HIST1:REF:LINK {tsco['det3']};:HIST1:STOP:LINK {tsco['det1']}")
        zmq_exec(tc, f"HIST2:REF:LINK {tsco['det4']};:HIST2:STOP:LINK {tsco['det2']}")
        zmq_exec(tc, f"HIST3:REF:LINK {tsco['det3']};:HIST3:STOP:LINK {tsco['det2']}")
        zmq_exec(tc, f"HIST4:REF:LINK {tsco['det4']};:HIST4:STOP:LINK {tsco['det1']}")
    
    except (ConnectionError, DataLinkTargetError, NotADirectoryError) as e:
        text_my_phone("Configuration Error", str(e))
        print(e)

def configure_tc_windows(window, input=4, windows=2, period=10000, width=4000, tau=cycle_time, invert=False):
    try:
        tc = connect(DEFAULT_TC_ADDRESS)
    
        dlt = dlt_connect(output_dir, DEFAULT_DLT_PATH)

        # default
        gen = "GEN" + tsco[input]
        tsc = "TSCO" + tsco[input]
        opin = "ONLYFIR"
        opou = "MUTE"

        if invert:
            opin = "MUTE"
            opou = "ONLYFIR"
        
        if window == -1:
            zmq_exec(tc, f"{gen}:ENAB OFF")
            zmq_exec(tc, f"{tsc}:WIND:ENAB OFF;:{tsc}:OPIN ONLYFIR;:{tsc}:OPOU ONLYFIR")
        else:
            delay = cyc2_delays[input] + tau*(window-2)
            zmq_exec(tc, f"{gen}:ENAB ON;:{gen}:PNUM {windows};:{gen}:PPER {period};:{gen}:PWID {width}")
            zmq_exec(tc, f"{gen}:TRIG:ARM:MODE AUTO;:{gen}:TRIG:DELA {delay};:{gen}:TRIG:LINK STAR")
            zmq_exec(tc, f"{tsc}:WIND:ENAB ON;:{tsc}:OPIN {opin};:{tsc}:OPOU {opou}")
            zmq_exec(tc, f"{tsc}:WIND:BEGI:DELA 0;:{tsc}:WIND:BEGI:EDGE RISI;:{tsc}:WIND:BEGI:LINK {gen}")
            zmq_exec(tc, f"{tsc}:WIND:END:DELA 0;:{tsc}:WIND:END:EDGE FALL;:{tsc}:WIND:END:LINK {gen}")
    
    except (ConnectionError, DataLinkTargetError, NotADirectoryError) as e:
        text_my_phone("Configuration Error", str(e))
        print(e)


In [4]:

 
def storeData(data, file_name, dir=output_dir):
    address = Path(str(dir) +'\\'+ str(file_name) + '.pkl')
    pkl_file = open(address, 'ab')
    
    # source, destination
    pickle.dump(data, pkl_file)                    
    pkl_file.close()
 
def loadData(file_name, dir=output_dir):
    address = Path(str(dir) +'\\'+ str(file_name) + '.pkl')
    pkl_file = open(address, 'rb')
       
    data = pickle.load(pkl_file)
    pkl_file.close()

    return data


def remove_duplicates(lst: list):
    # Removing duplicates using dictionary keys
    temp_list = list(dict.fromkeys(lst))
    lst.clear()
    lst.extend(temp_list)

# test_list = [4,3,3,4,5,6,7,8,8,8,9,4,3,7,5,4]
# print(test_list)
# remove_duplicates(test_list)
# print(test_list)


import sys
import logging

from pathlib import Path

utils_path = 'C:/Users/username/Documents/Research/Lab_Data/Automation_data_collection/ID900_Python_API_Scripts'   ##update##
# utils_path = Path('C:/Users/username/Downloads/TimeController_V1_11_0/Examples/Python')
sys.path.append(utils_path)
remove_duplicates(sys.path)
from utils.common import connect, zmq_exec, adjust_bin_width, dlt_connect, DataLinkTargetError, adjust_bin_width
from utils.acquisitions import acquire_histograms, save_histograms_commas, acquire_timestamps_and_histograms, close_active_acquisitions
from utils.plot import plot_histograms

logger = logging.getLogger(__name__)

# Default Time Controller IP address
DEFAULT_TC_ADDRESS = "169.254.103.125"

# Default acquisition duration in seconds
DEFAULT_ACQUISITION_DURATION = 60

# Default delay (in ps) to open the window after the tigger
DEFAULT_WINDOW_DELAY = 100000

# Default duration (in ps) of the opened window
DEFAULT_WINDOW_DURATION = 4000 # 4 ns

# Default bin_width (None = automatically set the lowest possible bin width)
DEFAULT_BIN_WIDTH = None

# Generate dummy signals on all inputs (no wire required)
DEMO_MODE = False

# Default file path where histograms are saved in CSV format (None = do not save)
DEFAULT_FILEPATH = Path("C:/Users/username/Documents/Research/Lab_Data/Automation_data_collection/TestOutputData/")   ##update##

# Default log file path where logging output is stored
DEFAULT_LOG_PATH = DEFAULT_FILEPATH / "Log/test_log.txt"

# Folder of the "DataLinkTargetService.exe" executable on your computer.
# Once the ID Quantique GUI is installed, you should find it there:
DEFAULT_DLT_PATH = Path("C:/Program Files/IDQ/Time Controller/packages/ScpiClient")


def configure_filtering(tc, window_delay, window_duration):
    # Configure input1 -> delay1 -> tsco5 (no filtering)
    zmq_exec(tc, "INPU1:ENAB ON;THRE -0.4V;COUP DC;EDGE FALLING;SELE UNSHAPED")
    zmq_exec(tc, "DELA1:VALUe 0;LINK INPU1")
    zmq_exec(tc, "TSCO5:WIND:ENAB OFF;:TSCO5:FIR:LINK DELA1;:TSCO5:OPOUt ONLYFIR")

    # Configure input2 -> delay2 -> tsco6 (without filtering)
    #                            -> tsco7 (with filtering)
    #           input3 -> delay3 -> tsco7 (as filter trigger)

    # input1 -> delay2
    zmq_exec(tc, "INPU2:ENAB ON;THRE -0.4V;COUP DC;EDGE FALLING;SELE UNSHAPED")
    zmq_exec(tc, "DELA2:VALUe 0;LINK INPU2")

    # input3 -> delay3
    zmq_exec(tc, "INPU3:ENAB ON;THRE -0.4V;COUP DC;EDGE FALLING;SELE UNSHAPED")
    zmq_exec(tc, "DELA3:VALUe 0;LINK INPU3")

    # delay2 -> tsco6 (no filtering)
    zmq_exec(tc, "TSCO6:WIND:ENAB OFF;:TSCO6:FIR:LINK DELA2;:TSCO6:OPOUt ONLYFIR")

    # delay2 -> tsco7 (with filtering)
    zmq_exec(tc, "TSCO7:WIND:ENAB ON;:TSCO7:FIR:LINK DELA2")
    # Setup when the window starts ('window_delay' ps after a rising edge event on INPU3)
    zmq_exec(tc, f"TSCO7:WIND:BEGI:LINK DELA3;DELAY {window_delay};EDGE RISING")
    # Setup when the window ends ('window_duration' ps after a rising edge event on INPU3)
    end_delay = window_delay + window_duration
    zmq_exec(tc, f"TSCO7:WIND:END:LINK DELA3;DELAY {end_delay};EDGE RISING")
    # Setup what the window does (let the signal from INPU2 through only inside the window)
    zmq_exec(tc, "TSCO7:OPIN ONLYFIR;:TSCO7:OPOUt MUTE")

    # histogram1: input1 as start (tsco5) and unfiltered input2 as stop (tsco6)
    zmq_exec(tc, "HIST1:REF:LINK TSCO5;:HIST1:STOP:LINK TSCO6")

    # histogram2: input1 as start (tsco5) and filtered input2 as stop (tsco7)
    zmq_exec(tc, "HIST2:REF:LINK TSCO5;:HIST2:STOP:LINK TSCO7")


def configure_dummy_signals(tc):
    ## Configure start signal (trigger) and link it to both output 1 and 3
    zmq_exec(tc, "GEN1:ENAB OFF")
    zmq_exec(tc, "GEN1:PPER 4000000;PWID 4000;PNUM INF;TRIG:ARM:MODE MANUal")
    zmq_exec(tc, "GEN1:ENAB ON;PLAY")

    zmq_exec(tc, "TSCO1:WIND:ENAB OFF;:TSCO1:FIR:LINK GEN1;:TSCO1:OPOUt ONLYFIR")
    zmq_exec(tc, "OUTP1:ENAB ON;MODE NIM;LINK TSCO1")

    ## Configure stop signal and link it to output 2
    zmq_exec(tc, "GEN2:ENAB OFF")
    zmq_exec(tc, "GEN2:PPER 4000100;PWID 4000;PNUM INF;TRIG:ARM:MODE MANUal")
    zmq_exec(tc, "GEN2:ENAB ON;PLAY")
    zmq_exec(tc, "TSCO2:WIND:ENAB OFF;:TSCO2:FIR:LINK GEN2;:TSCO2:OPOUt ONLYFIR")
    zmq_exec(tc, "OUTP2:ENAB ON;MODE NIM;LINK TSCO2")

    zmq_exec(tc, "OUTP3:ENAB ON;MODE NIM;LINK TSCO1")  # output start as trigger signal

    zmq_exec(tc, "INPU1:SELE OUTP")
    zmq_exec(tc, "INPU2:SELE OUTP")
    zmq_exec(tc, "INPU3:SELE OUTP")



import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Email and password for the sending email account
email = "youremail@mail.com"   ##update##
password = "password"    ##update##

# Recipient phone number and carrier's SMS gateway domain
# AT&T: {number}@txt.att.net
# Verizon: {number}@vtext.com
# T-Mobile: {number}@tmomail.net
# Sprint: {number}@messaging.sprintpcs.com
phone_number = "5555555555"   ##update##
# sms_gateway = f"{phone_number}@vtext.com"  # Replace with your carrier's domain

def text_my_phone(subject: str, body: str, 
                  phone_number=phone_number,
                  carrier_domain="@vtext.com", # Replace with your carrier's domain   ##update##
                  email=email, 
                  password=password
                 ):

    sms_gateway = phone_number + carrier_domain
    
    # Create the email
    msg = MIMEMultipart()
    msg['From'] = email
    msg['To'] = sms_gateway
    msg['Subject'] = subject
    time_stamp = datetime.now().strftime("%H:%M %d %b")
    text_body = "sent at " + time_stamp + "\n" + body + "\n*"
    msg.attach(MIMEText(text_body, 'plain'))
    
    # Connect to the Gmail SMTP server and send the email
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(email, password)
    text = msg.as_string()
    server.sendmail(email, sms_gateway, text)
    server.quit()
    


tsco = {"det1": "TSCO5", 
        "det2": "TSCO6", 
        "det3": "TSCO7", 
        "det4": "TSCO8",
             1: "5",
             2: "6",
             3: "7",
             4: "8"}


delay1 = 322500
delay2 = 320000
hist_center = 100000 # histograms centered at 100 ns
cycle_time = 27000
window_delay4 = 112000 # Beginning of first window for 2 round trips
window_delay3 = window_delay4
window_delay2 = window_delay4 + hist_center
window_delay1 = window_delay2

cyc2_delays = {1: window_delay1,
               2: window_delay2,
               3: window_delay3,
               4: window_delay4}

def configure_tc_CQM1_triggers():
    try:
        tc = connect(DEFAULT_TC_ADDRESS)
    
        dlt = dlt_connect(output_dir, DEFAULT_DLT_PATH)
        
        zmq_exec(tc, f"DELA1:VALUe {delay1-2*hist_center}")
        zmq_exec(tc, f"DELA2:VALUe {delay2-2*hist_center}")
    
        # histogram1: input1 as start (tsco5) and unfiltered input2 as stop (tsco6)
        zmq_exec(tc, f"HIST1:REF:LINK {tsco['det1']};:HIST1:STOP:LINK {tsco['det3']}")
        zmq_exec(tc, f"HIST2:REF:LINK {tsco['det2']};:HIST2:STOP:LINK {tsco['det4']}")
        zmq_exec(tc, f"HIST3:REF:LINK {tsco['det2']};:HIST3:STOP:LINK {tsco['det3']}")
        zmq_exec(tc, f"HIST4:REF:LINK {tsco['det1']};:HIST4:STOP:LINK {tsco['det4']}")
    
    except (ConnectionError, DataLinkTargetError, NotADirectoryError) as e:
        text_my_phone("Configuration Error", str(e))
        print(e)

def configure_tc_CQM2_triggers():
    try:
        tc = connect(DEFAULT_TC_ADDRESS)
    
        dlt = dlt_connect(output_dir, DEFAULT_DLT_PATH)
        
        zmq_exec(tc, f"DELA1:VALUe {delay1}")
        zmq_exec(tc, f"DELA2:VALUe {delay2}")
    
        # histogram1: input1 as start (tsco5) and unfiltered input2 as stop (tsco6)
        zmq_exec(tc, f"HIST1:REF:LINK {tsco['det3']};:HIST1:STOP:LINK {tsco['det1']}")
        zmq_exec(tc, f"HIST2:REF:LINK {tsco['det4']};:HIST2:STOP:LINK {tsco['det2']}")
        zmq_exec(tc, f"HIST3:REF:LINK {tsco['det3']};:HIST3:STOP:LINK {tsco['det2']}")
        zmq_exec(tc, f"HIST4:REF:LINK {tsco['det4']};:HIST4:STOP:LINK {tsco['det1']}")
    
    except (ConnectionError, DataLinkTargetError, NotADirectoryError) as e:
        text_my_phone("Configuration Error", str(e))
        print(e)

def configure_tc_windows(window, input=4, windows=2, period=10000, width=4000, tau=cycle_time, invert=False):
    try:
        tc = connect(DEFAULT_TC_ADDRESS)
    
        dlt = dlt_connect(output_dir, DEFAULT_DLT_PATH)

        # default
        gen = "GEN" + tsco[input]
        tsc = "TSCO" + tsco[input]
        opin = "ONLYFIR"
        opou = "MUTE"

        if invert:
            opin = "MUTE"
            opou = "ONLYFIR"
        
        if window == -1:
            zmq_exec(tc, f"{gen}:ENAB OFF")
            zmq_exec(tc, f"{tsc}:WIND:ENAB OFF;:{tsc}:OPIN ONLYFIR;:{tsc}:OPOU ONLYFIR")
        else:
            delay = cyc2_delays[input] + tau*(window-2)
            zmq_exec(tc, f"{gen}:ENAB ON;:{gen}:PNUM {windows};:{gen}:PPER {period};:{gen}:PWID {width}")
            zmq_exec(tc, f"{gen}:TRIG:ARM:MODE AUTO;:{gen}:TRIG:DELA {delay};:{gen}:TRIG:LINK STAR")
            zmq_exec(tc, f"{tsc}:WIND:ENAB ON;:{tsc}:OPIN {opin};:{tsc}:OPOU {opou}")
            zmq_exec(tc, f"{tsc}:WIND:BEGI:DELA 0;:{tsc}:WIND:BEGI:EDGE RISI;:{tsc}:WIND:BEGI:LINK {gen}")
            zmq_exec(tc, f"{tsc}:WIND:END:DELA 0;:{tsc}:WIND:END:EDGE FALL;:{tsc}:WIND:END:LINK {gen}")
    
    except (ConnectionError, DataLinkTargetError, NotADirectoryError) as e:
        text_my_phone("Configuration Error", str(e))
        print(e)




def tc_check_start_signals():
    try:
        tc = connect(DEFAULT_TC_ADDRESS)
        dlt = dlt_connect(output_dir, DEFAULT_DLT_PATH)

        resp = zmq_exec(tc, "STAR:COUN?")

        return int(resp)
    
    except (ConnectionError, DataLinkTargetError, NotADirectoryError) as e:
        # text_my_phone("Configuration Error", str(e))
        print(e)


import sys
import time
import numpy as np

def visual_timer(duration, flash_speed=2, update_interval=0.1, max_size=20):
    '''flash_speed=2 number of flashes per second
    update_interval=0.1 seconds between updating'''
    
    total_dashes = int( min(max_size, duration) )

    # Decrease update interval if flash_speed is faster
    update_interval = min(update_interval, 1/flash_speed)
    
    # Print initial dashes and return to start without new line
    print('-' * total_dashes, end='', flush=True)
    sys.stdout.flush()
 

    # Record keeping variables
    start_time = time.time()
    elapsed_time = time.time() - start_time
    onum = 0; # number of o's
    isFlashed = False;
    
    while elapsed_time < duration:
        percentage_complete = elapsed_time / duration
        dashes_to_overwrite = int(percentage_complete * total_dashes)
        
        # For flashspeed=2 it returns false for # <= time < #.5 and true #.5 <= time < #+1
        # That means it should flash every 0.5 seconds regardless of faster update speeds
        shouldFlash = np.mod(int(elapsed_time*flash_speed),2)

        # Need to update if there are more dashes to overwrite or we need to change the flashing dash
        if dashes_to_overwrite - onum or shouldFlash != isFlashed:
            overwrite = '\r' + 'o' * dashes_to_overwrite
            onum = dashes_to_overwrite

            if shouldFlash:
                overwrite += ' '
                isFlashed = True
            else:
                overwrite += '-'
                isFlashed = False

            # Return to the start of the line and overwrite with overwrite string
            sys.stdout.write(overwrite)
            sys.stdout.flush()
        

        # Wait for a bit
        time.sleep(update_interval)
        
        # Update elapsed_time
        elapsed_time = time.time() - start_time

    # Overwrite the line with 'o's at the end to ensure complete overwrite
    sys.stdout.write('\r' + 'o' * total_dashes + '\n')
    sys.stdout.flush()
    

#visual_timer(40, flash_speed=4)
#visual_timer(68)



def run_ID900_aquisition(
    duration=60, 
    address="169.254.103.125",
    windowed=False, 
    window_delay=DEFAULT_WINDOW_DELAY,
    window_duration=4000, 
    bin_width=None, 
    bin_count=16384,
    hist_numbers=[1,2,3,4],
    hist_names={1:"Hist 1",2:"Hist 2",3:"Hist 3",4:"Hist 4"},
    directory=DEFAULT_FILEPATH,
    timestamp_filename="timestamps",
    histogram_filename="",
    with_ref_index=True,
    format="bin",
    plotting=True,
    loggingBool=False,
    log_path=DEFAULT_LOG_PATH,
    verbose_logging=False,
    dlt_exe_path=DEFAULT_DLT_PATH
):
    
    logging.basicConfig(
        level=logging.DEBUG if verbose_logging else logging.INFO,
        format="%(levelname)s: %(message)s",
        filename=log_path if loggingBool else None
    )

    success = True
    hists = {}
    
    try:
        tc = connect(address)

        dlt = dlt_connect(directory, dlt_exe_path)

        bin_width = adjust_bin_width(tc, bin_width)

        # Close any ongoing acquisition on the DataLinkTarget
        close_active_acquisitions(dlt)

        [success, histograms] = acquire_timestamps_and_histograms(
            tc,
            dlt,
            address,
            duration,
            hist_numbers,
            format,
            directory,
            timestamp_filename,
            with_ref_index, 
            bin_width, 
            bin_count,
        )

        hists = {key if hist_names is None else hist_names[key]: vals for key, vals in histograms.items()}

        if histogram_filename:
            save_histograms_commas(hists, bin_width, directory / histogram_filename)

        if plotting:
            plot_histograms(hists, bin_width)

    except (ConnectionError, DataLinkTargetError, NotADirectoryError) as e:
        text_my_phone("ID900 Aquisition Error", str(e))
        print(e)
        success = False

    return hists
    # exit(0 if success else 1)

    


import requests
import numpy as np


QM1_BEFORE = 1
QM1_AFTER = 2
QM2_BEFORE = 3
QM2_AFTER = 4

DEGREES_PER_STEP = .45 * 20/60 # =0.15 


class Stepper:
    def __init__(self, id, url, driver_location, steps_per_full_rotation=800, position=0, rotation_direction=0, reverse_facing=False):
        self.id = id
        self.url = url
        self.driver = driver_location # this is the driver on the CNC shield to control (i.e. 'X', 'Y', 'Z', or 'A')
        self.steps_per_full_rotation = steps_per_full_rotation
        self.ratio = steps_per_full_rotation/360 # ratio is steps per degree rotation
        self.pos = np.mod(position,360)
        self.dir = np.sign(rotation_direction) # -1: only rotate in the neg, 1: only rotate in the pos, 0: rotate with given direction
        self.rev_face = reverse_facing # If true, the polarizer is facing the opposite direction so we flip the sign when updating the position

    def step(self, steps, do_print=False, send_error=False):
        # Check if it is asking to go the wrong direction (neg*pos and pos*neg < 0)
        if self.dir == 0:
            steps = round(np.mod(steps, np.sign(steps) * self.steps_per_full_rotation))
        else:
            steps = round(np.mod(steps, self.dir * self.steps_per_full_rotation))

        if steps == 0:
            if do_print: print(f"Polarizer {self.id} doesn't need to rotate. Position: {self.pos}°")
            return
        
        url = f"{self.url}/{self.driver}{steps}"
        if do_print: print("\n",url)
        try:
            response = requests.get(url +'\n')
            if response.status_code == 200:
                self.pos = np.mod(self.pos + (-1)**self.rev_face * steps/self.ratio, 360)
                if do_print: print(f"Polarizer {self.id} rotated successfully! Position: {self.pos}°")
            else:
                if do_print: print(f"Failed to step polarizer {self.id}.")
        except Exception as e:
            print(f"\n Motor Error occurred for motor {self.id}: {e}")
            if send_error: text_my_phone(f"Motor {self.id} Error", str(e))

    
    def rotate(self, degrees, do_print=False, send_error=False):        
        steps = round(degrees*self.ratio) *(-1)**self.rev_face # 
        self.step(steps, do_print=do_print)

    
    def toPosition(self, angle, do_print=False, send_error=False):
        # Find shortest path to desired angle
        rotate = np.mod(angle,360) - self.pos
        if rotate > 180:
            rotate -= 360
        self.rotate(rotate, do_print=do_print)

    def setPos(self, angle):
        self.pos = angle;


# Set the base URL to your Arduino's IP address
CQM1_URL = "http://10.200.72.68"   ##update##
CQM2_URL = "http://10.200.141.211"   ##update##

# multiply 800 by 3 (gear ratio 20:60)
motor1 = Stepper(1, CQM1_URL, 'X', steps_per_full_rotation=800*3, rotation_direction=1, reverse_facing=True)
motor2 = Stepper(2, CQM1_URL, 'Y', steps_per_full_rotation=800*3, rotation_direction=-1)
motor3 = Stepper(3, CQM2_URL, 'X', steps_per_full_rotation=800*3, rotation_direction=1, reverse_facing=True)
motor4 = Stepper(4, CQM2_URL, 'Y', steps_per_full_rotation=800*3, rotation_direction=-1)


def printPositions():
    print(f'Pol1: {motor1.pos}°')
    print(f'Pol2: {motor2.pos}°')
    print(f'Pol3: {motor3.pos}°')
    print(f'Pol4: {motor4.pos}°')

# Function to convert binary to decimal
def binary_to_decimal(binary_str):
    # Convert the binary string to a decimal integer
    decimal_number = int(binary_str, 2)
    return decimal_number

def read_binary_IPAdresses():
    while True:
        # Input binary numbers separated by periods from the user
        binary_input = input("Enter 8-bit binary numbers separated by a '.' or 'exit': ")
    
        if 'exit' in binary_input or binary_input == "":
            break
        
        # Split the input string by periods
        binary_numbers = binary_input.split('.')
        
        # Check each binary number and convert to decimal
        decimal_numbers = []
        valid_input = True
        
        for binary in binary_numbers:
            if len(binary) <= 8 and all(bit in '01' for bit in binary):
                decimal_numbers.append(binary_to_decimal(binary))
            else:
                valid_input = False
                print(f"Invalid 8-bit binary number: {binary}")
                break
        
        # Output the decimal numbers if all inputs are valid
        if valid_input:
            decimal_output = '.'.join(map(str, decimal_numbers))
            print(f"The decimal form of {binary_input} is {decimal_output}.")


h = 0
v = 90
d = 45
a = 135

H = 180
V = 270
D = 225
A = 315

   ##update##
pol1_offset = 169.5
pol2_offset = 56.5
pol3_offset = 149
pol4_offset = 96

motors = [motor1, motor2, motor3, motor4]

def move_motors_with_offset(pos1,pos2,pos3,pos4,printPos=False):
    motor1.toPosition(pos1 + pol1_offset)
    motor2.toPosition(pos2 + pol2_offset)
    motor3.toPosition(pos3 + pol3_offset)
    motor4.toPosition(pos4 + pol4_offset)

    if printPos:
        print(f'Pol1: {motor1.pos - pol1_offset}°')
        print(f'Pol2: {motor2.pos - pol2_offset}°')
        print(f'Pol3: {motor3.pos - pol3_offset}°')
        print(f'Pol4: {motor4.pos - pol4_offset}°')

def move_motors(pos: list[int], printPos=False, send_errors=False):
    for i in range(len(pos)):
        motors[i].toPosition(pos[i], send_error=send_errors)

    if printPos:
        for i in range(len(pos)):
            print(f'Pol{i+1}: {motors[i].pos}°')


def roundd(value, resolution=1):
    rval = np.round(value / resolution) * resolution
    if isinstance(resolution, int):
        rval = int(rval)
    return rval
    
        

print("done")

done


In [5]:
# configure_tc_windows(3, input=4)
# configure_tc_windows(0, input=3)
# configure_tc_windows(-1, input=2)
# configure_tc_windows(-1, input=1)

In [6]:
tc_check_start_signals()

0

In [86]:
read_binary_IPAdresses()

Enter 8-bit binary numbers separated by a '.' or 'exit':  10001101.01101101


The decimal form of 10001101.01101101 is 141.109.


Enter 8-bit binary numbers separated by a '.' or 'exit':  


In [7]:
# Update IP addresses
CQM1_URL = "http://10.200.141.109"   ##update##
CQM2_URL = "http://10.200.9.152"   ##update##

motor1.url = CQM1_URL
motor2.url = CQM1_URL
motor3.url = CQM2_URL
motor4.url = CQM2_URL

# print(f"Motor {motor1.id}'s ip address: {motor1.url}")
# print(f"Motor {motor2.id}'s ip address: {motor2.url}")
# print(f"Motor {motor3.id}'s ip address: {motor3.url}")
# print(f"Motor {motor4.id}'s ip address: {motor4.url}")

In [8]:
print(motor2.url)
print(motor3.url)

http://10.200.141.109
http://10.200.9.152


In [9]:
# set motor positions if needed
motor1.pos = 45
motor2.pos = 45
motor3.pos = h
motor4.pos = h
printPositions()

Pol1: 45°
Pol2: 45°
Pol3: 0°
Pol4: 0°


In [10]:
move_motors([d,d,d,d], send_errors=True)
printPositions()

Pol1: 45°
Pol2: 45°
Pol3: 45.0°
Pol4: 45.0°


In [11]:
printPositions()

Pol1: 45°
Pol2: 45°
Pol3: 45.0°
Pol4: 45.0°


In [12]:
import serial
        
class Cerial:
    def __init__(self, port, baud_rate=38400, timeout=1, termination='\r\n'):
        
        self.ser = serial.Serial(
            port=port,
            baudrate=baud_rate,
            bytesize=serial.EIGHTBITS,
            parity=serial.PARITY_NONE,
            stopbits=serial.STOPBITS_ONE,
            timeout=timeout
            )

        try:
            self.termination = termination
            self.success = False
            self.resp = 'n/a'
            self.device = 'none'
            
            self.try_query("*IDN?", tries=2)
            
            if self.success == False:
                #try the other baud_rate
                self.ser.baudrate = 115200 if baud_rate == 38400 else 38400
                self.try_query("*IDN?", tries=2)
                if self.success == True:
                    print(f"Connected using baud rate: {self.ser.baudrate}")
            
            if self.resp == '?1':
                time.sleep(0.5)
                self.try_query("*IDN?", tries=2)
    
            if self.success == True and self.resp != '?1':
                self.device = self.resp
                print(f"Connected to {self.device}")
            else:
                print(f"Failed to initialize Cerial: {self.resp}")
                print(f"success: {self.success}  resp: {self.resp}")
                self.__del__()

        except Exception as e: 
            self.ser.close()
            print('flag1')
            raise e

    

    def __del__(self):
        self.ser.close()

    def open(self):
        if not self.ser.is_open:
            self.ser.open()

    def close(self):
        if self.ser.is_open:
            self.ser.close()
    
    def write(self, command):
        self.open()
        command += self.termination  # Append termination characters to the command
        self.ser.write(command.encode())
    
    def read(self):
        self.open()
        self.resp = self.ser.readline().decode().strip()
        self.close()
        if self.resp == '':
            raise Exception('Read Timed Out')
        return self.resp
    
    def query(self, command, query_delay=0):
        self.write(command)
        time.sleep(query_delay)
        return self.read()

    def try_read_again(self, exception, tries=1, delay=1, prnt=False, first=True):
        if 'Read Timed Out' in str(exception):
            
            if prnt and first: print(f"Command timed out, trying again... ", end='')
            elif prnt: print(f"\n      and again (tries left: {tries-1})... ", end='')
                
            time.sleep(delay)
            
            try: 
                resp = self.read()
                
            except Exception as e: 
                
                if tries > 1:
                    resp = self.try_read_again(e, tries=tries-1, delay=delay, prnt=prnt, first=False)
                    return resp
                else:
                    if prnt: print('nope')
                    return [False, str(e)]
                    
            else: 
                if prnt: print('Success!')
                return [True, resp]
                
        else:
            if prnt: print('nope')
            return [False, str(exception)]
    
    
    def try_query(self, command, send_errors=False, prnt=False, tries=1):
        try:
            self.resp = self.query(command)
            self.success = True
            if prnt: print(f"{command} -> {self.resp}")
            
        except Exception as e:
            if prnt: print('')
            [self.success, self.resp] = self.try_read_again( e, prnt=prnt, tries=tries)
            if self.success:
                if prnt: print(f"{command} -> {self.resp}")
            else:
                if prnt: print(f"ERROR Query: {command}, Error: {self.resp}")
                if send_errors: text_my_phone("Serial Query Error", self.resp)
                  

    
def set_Cycles_BNC(bnc, cqm, cycles, cyc_time=26, send_errors=False, prnt=False, turnOffFirst=False):
    commands = [':SPULSE:STATE OFF'] if turnOffFirst else []
        
    match cycles:
        case 0:
            commands += [f":PULSE{cqm}:STATE OFF"] 
            commands += [f":PULSE{cqm}:POL INVERTED"]
        case 1:
            commands += [f":PULSE{cqm}:STATE OFF"] 
            commands += [f":PULSE{cqm}:POL NORM"]
        case _:
            commands += [f":PULSE{cqm}:STATE ON"] 
            commands += [f":PULSE{cqm}:POL NORM"]
            commands += [f":PULSE{cqm}:WIDTH {roundd(cyc_time*(cycles-1), 0.25)}e-9"]

    if turnOffFirst: commands += [':SPULSE:STATE ON']
        
    for command in commands:
        bnc.try_query(command, send_errors=send_errors, prnt=prnt)


import re

def get_fir_word_level_and_sec_word(string):
    match1 = re.search(r"\b\S+?\b", string)
    match2 = re.search(r"\b\S+?$", string)
    if match1 is None or match2 is None:
        return [None, 0, None]
    else:
        word1 = match1[0]
        lvl = int(match1.span()[0] / 4)
        word2 = match2[0]
        return [word1, lvl, word2]


unaccepted_strings = ['count', 'clear', 'merror', 'iwidth', 'idelay']

def populate_dict_at_lvl(bnc, strLst, i, length, cur_lvl=1, key=':', num=8):
    return_dict = {}
    
    while i < length:
        [word, lvl, typ] = get_fir_word_level_and_sec_word(strLst[i])
        
        if lvl == 0:
            i+=1
            
        elif lvl < cur_lvl:
            break
            
        elif typ == 'Subsystem':
            # print(f"SubSystem: {typ}, word: {word}")
            [i, return_dict[word]] = populate_dict_at_lvl(bnc, strLst, i+1, length, cur_lvl+1, num=num, key=f"{key}{word}:")
            
        elif typ == 'SubSystem#':
            # print(f"SubSystem#: {typ}, word: {word}")
            for j in range(num):
                wordnum = f"{word}{j+1}"
                [k, return_dict[wordnum]] = populate_dict_at_lvl(bnc, strLst, i+1, length, cur_lvl+1, num=num, key=f"{key}{wordnum}:")
            i=k
                
        elif typ != "Query" and word.lower() not in unaccepted_strings:
            query = f"{key}{word}?"
            # print(f"Non Query: {typ}, word: {word}, query: {query}")
            bnc.try_query(query)

            if bnc.success:
                if bnc.resp[0] == '?':
                    print(f"Query error for {query} -> {bnc.resp}")
                    return_dict[word] = None
                else:
                    return_dict[word] = bnc.resp
            else:
                print(f"Query: {query}, Error: {bnc.resp}")
                return_dict[word] = None
                
            i+=1
            
        else:
            i+=1


    return [i, return_dict]
            
    

def get_and_parse_BNC_SCPI_command_tree_into_dict(bnc):

    idn = bnc.device

    num_pulses = re.search(r"(?<=575-)\d*", idn)
    num_pulses = int(num_pulses[0])

    bnc.try_query(":INST:COMM?", prnt=True)
    if bnc.success:
        command_tree = bnc.resp
    else:
        return            
        
    strList = command_tree.split("\r")

    i = 0
    length = len(strList)
    [i, dict] = populate_dict_at_lvl(bnc, strList, i, length, num=num_pulses)

    print('Done')
    
    return dict


def recursive_print_dict(dict, lvl=0, first=True):
    frst = first
    try:
        for key, val in dict.items():
            
            if frst: 
                print(f"{'    '*lvl}{key.upper()}:", end='')
                frst = False
            else: 
                print(f"\n{'    '*lvl}{key.upper()}:", end='')
                
            recursive_print_dict(val, lvl=lvl+1, first=False)
            
    except Exception as e:
        print(f" {dict}", end='')

unaccepted_commands = unaccepted_strings + ['system','nselect','select','{','}']

def contains_any(target_string, list_of_strings, case_sensitive=False):
    for substring in list_of_strings:
        if substring in target_string or (substring.lower() in target_string.lower() and not case_sensitive):
            return True
    return False

def recursive_set_BNC_Generator(bnc, dict, command):
    try:
        for key, val in dict.items():
            recursive_set_BNC_Generator(bnc, val, f"{command}:{key}")
            
    except:
        if dict is None: return
        if contains_any(command, unaccepted_commands): return

        bnc.try_query(f"{command} {dict}")

        if bnc.success:
            if bnc.resp[0] == '?':
                print(f"Error: {command} {dict} -> {bnc.resp}")
            else:
                print(f"{command} {dict} -> {bnc.resp}")
        else:
            print(f"Command: {command} {dict}, Error: {bnc.resp}")



In [13]:
bnc_cer = Cerial('COM8')

Connected to BNC,575-8,35550,2.4.2-2.0.11


In [36]:
# bnc_cer.try_query(':SPULSE:STATE ON')

In [14]:
bnc_cer.query(':SPULSE:STATE?')

'0'

In [15]:
tc_check_start_signals()

0

In [17]:
move_motors([a,a,d,d], send_errors=True)
printPositions()

Pol1: 135.0°
Pol2: 135.0°
Pol3: 45.0°
Pol4: 45.0°


In [20]:
# motor1.pos = 135
# motor2.pos = 135
# motor3.pos = 135
# motor4.pos = 135
# printPositions()

In [52]:
printPositions()

Pol1: 45°
Pol2: 135.0°
Pol3: 45°
Pol4: 135.0°


In [73]:
pol_settings = {}

# angles1 = {'45': d,
#            '225': D,
#            'h': h,
#            'H': H}
angles1 = {'0': h,
           '45': d}

for key1, angle1 in angles1.items():
    for angle2 in range(360,-1,-45):
        pol_settings[f"{angle2},{angle2},{angle1},{angle1}"] = [angle2,angle2,angle1,angle1]
        if angle2-18 > 0: 
            pol_settings[f"{angle2-18},{angle2-18},{angle1},{angle1}"] = [angle2-18,angle2-18,angle1,angle1]

In [57]:
output_file_names = {}
histogram_data = {}

In [29]:
##########################################
### Main code that runs the experiment ###
##########################################

h = 0
v = 90
d = 45
a = 135

output_file_names = {}
histogram_data = {}

pol_settings = {}

# angles1 = {'45': d,
#            '225': D,
#            'h': h,
#            'H': H}
# angles1 = {'90': v,
#            '270': V}
# angles1 = {'45': d}#,
#            # '225': D}

# for key1, angle1 in angles1.items():
#     for angle2 in range(360,-1,-45):
#         # if angle2+18 < 360: 
#         #     pol_settings[f"{angle1},{angle2+18},{angle1},{angle2+18}"] = [angle1,angle2+18,angle1,angle2+18]
#         pol_settings[f"{angle1},{angle2},{angle1},{angle2}"] = [angle1,angle2,angle1,angle2]
#         if angle2-18 > 0: 
#             pol_settings[f"{angle1},{angle2-18},{angle1},{angle2-18}"] = [angle1,angle2-18,angle1,angle2-18]

angles1 = {'0': h,
           '45': d}

for key1, angle1 in angles1.items():
    for angle2 in range(360,-1,-45):
        pol_settings[f"{angle2},{angle2},{angle1},{angle1}"] = [angle2,angle2,angle1,angle1]
        if angle2-18 > 0: 
            pol_settings[f"{angle2-18},{angle2-18},{angle1},{angle1}"] = [angle2-18,angle2-18,angle1,angle1]
               

output_channels = [{1: "b2-b1", 2: "a2-a1", 3: "b2-a1", 4: "a2-b1"}]#,
                   # {1: "b1-b2", 2: "a1-a2", 3: "a1-b2", 4: "b1-a2"}]

# dd: diaganol photons sent into both CQMs
# hv: horizontal and vertical photons sent into CQM-1 and 2 respectively
# ee#: photons are entangled in polarization and CQM-# is the one to change pol each time
trials = ["ee1"]#,"ee2"]#,"dd2","dd3"]

# cycle_settings = [[0,0],[0,1],[0,2],[0,3],
#                   [1,0],[1,1],[1,2],[1,3],
#                   [2,0],[2,1],[2,2],[2,3],
#                   [3,0],[3,1],[3,2],[3,3]]
cycle_settings = [[0,0],[1,1],[2,2],[0,1]]
# cycle_settings = [[2,0],[2,1],[2,2],[2,3],
#                   [3,0],[3,1],[3,2],[3,3],
#                   [1,0],[1,1],[1,2],[1,3],
#                   [0,0],[0,1],[0,2],[0,3]]
# cycle_settings = [[0,0],[1,1],[2,2],[3,3]]



cyc_set_len = len(cycle_settings)

### Given an expected efficiency per cycle and the acquisition time (in seconds) for 0-0 cycles,
### this estimates the other acquisition times to maintain similar maximum coincidence counts (rounded to 30s)
expEff = 0.8
dur0 = 120*2
durs = list(map(lambda tuple: roundd(dur0/expEff**np.sum(tuple), 30), cycle_settings))
# durs = [120,180,240,300]

acquisition_num = cyc_set_len * len(pol_settings) * len(output_channels) * len(trials)
ave_acq_time = timedelta(seconds=np.average(durs))
correction = 7 # initial guess for the extra time to rotate polarizers between acquisitions 

### Initize metadata
t_start = datetime.now()
run_time = timedelta(seconds=acquisition_num * (ave_acq_time.total_seconds() + correction))
eta = t_start + run_time
count = 0
est_time_past =  timedelta(seconds=0)

t_start_str = t_start.strftime("%H:%M")
run_time_str = run_time
eta_str = eta.strftime("%H:%M")
print(f"Started at {t_start_str}")
print(f"Estimated run time: {run_time_str} (eta: {eta_str})")


bnc_cer.try_query(':SPULSE:STATE ON')

for trial in trials:
    
    for cyc, dur in zip(cycle_settings, durs):
        
        t_left = eta - datetime.now()
        days = t_left.days
        hrs = t_left.seconds // 3600
        mins = (t_left.seconds % 3600) // 60
        t_left_str = ""
        if days == 1:
            t_left_str += f"{days} day, "
        elif days > 1:
            t_left_str += f"{days} days, "
        t_left_str += f"{hrs}:{0 if mins<10 else ''}{mins}"
            
        text_my_phone(
            "Data Collection Update", 
            f" \nBeginning {cyc[0]}-{cyc[1]}\nEstimated time left: {t_left_str}\neta: {eta_str}\n "
        )
        
        set_Cycles_BNC(bnc_cer, 1, cyc[0], send_errors=True, turnOffFirst=True)
        set_Cycles_BNC(bnc_cer, 2, cyc[1], send_errors=True, turnOffFirst=True)

        # configure_tc_windows(-1, input=4)
        # configure_tc_windows(-1, input=3)
        configure_tc_windows(cyc[1], input=4)
        configure_tc_windows(0, input=3)
        configure_tc_windows(-1, input=2)
        configure_tc_windows(-1, input=1)
    
        time.sleep(2)
        
        start_signals_per_sec = tc_check_start_signals()
    
        if start_signals_per_sec < 3330000:
            
            if start_signals_per_sec < 1000:
                print(f"ERROR: only {start_signals_per_sec} start signals per second")
                text_my_phone("ERROR", f"only {start_signals_per_sec} start signals per second")
                time.sleep(2)
                break
                
            print(f"\nWARNING: only {start_signals_per_sec} start signals per second")
            text_my_phone("WARNING", f"only {start_signals_per_sec} start signals per second")

    
    
    
        for key, val in pol_settings.items():
            for chan in output_channels:

                count += 1

                ### Use the next line to pick up where you left off if needed ###
                # if count < 9: continue
                
                ouput_file_name = f"cyc({cyc[0]}-{cyc[1]})_tri({trial})_pol({key})_dur({dur})"
    
                key_name = f"{cyc[0]}-{cyc[1]},{trial},{key}"
                
                output_file_names[key_name] = ouput_file_name
    
                move_motors(val, send_errors=True)
    
                
                print(f"\n\n{ouput_file_name}  {count}/{acquisition_num} (eta: {eta_str})")
    
                histogram_data[key_name] = run_ID900_aquisition(duration=dur, bin_width=100, bin_count=2000, 
                                     hist_names=chan,
                                     directory=output_dir, 
                                     timestamp_filename=ouput_file_name,
                                     histogram_filename=ouput_file_name+".csv",
                                     plotting=False, loggingBool=True)
    
                
                time_past = datetime.now() - t_start
                est_time_past += timedelta(seconds=dur)

                # should be true that: time_past == est_time_past + correction*count
                correction = (time_past - est_time_past)/count
          
                eta = t_start + (acquisition_num * (ave_acq_time + correction))
                eta_str = eta.strftime("%H:%M")

    ### This repopulates the polarization settings so now CQM-2 is 
    pol_settings = {}
    for key1, angle1 in angles1.items():
        for angle2 in range(360,-1,-45):
            pol_settings[f"{angle1},{angle1},{angle2},{angle2}"] = [angle1,angle1,angle2,angle2]
            if angle2-18 > 0: 
                pol_settings[f"{angle1},{angle1},{angle2-18},{angle2-18}"] = [angle1,angle1,angle2-18,angle2-18]


    
text_my_phone("Data Collection Complete", "")
t_end = datetime.now().strftime("%H:%M")
print(f"\nDone: {t_end}")

print(f"Final Correction: {correction}")
bnc_cer.try_query(':SPULSE:STATE OFF')

Started at 16:41
Estimated run time: 14:25:52 (eta: 07:07)


cyc(0-0)_tri(ee1)_pol(180,180,0,0)_dur(240)  9/136 (eta: 07:07)
Histogram and Timestamp Aquisition Started (240s)
oooooooooooooooooooooooooooooo
Aquisition Complete


cyc(0-0)_tri(ee1)_pol(162,162,0,0)_dur(240)  10/136 (eta: 06:53)
Histogram and Timestamp Aquisition Started (240s)
oooooooooooooooooooooooooooooo
Aquisition Complete


cyc(0-0)_tri(ee1)_pol(135,135,0,0)_dur(240)  11/136 (eta: 06:54)
Histogram and Timestamp Aquisition Started (240s)
oooooooooooooooooooooooooooooo
Aquisition Complete


cyc(0-0)_tri(ee1)_pol(117,117,0,0)_dur(240)  12/136 (eta: 06:55)
Histogram and Timestamp Aquisition Started (240s)
oooooooooooooooooooooooooooooo
Aquisition Complete


cyc(0-0)_tri(ee1)_pol(90,90,0,0)_dur(240)  13/136 (eta: 06:56)
Histogram and Timestamp Aquisition Started (240s)
oooooooooooooooooooooooooooooo
Aquisition Complete


cyc(0-0)_tri(ee1)_pol(72,72,0,0)_dur(240)  14/136 (eta: 06:56)
Histogram and Timestamp Aquisition Star

In [25]:
set_Cycles_BNC(bnc_cer, 1, 0, send_errors=True, turnOffFirst=True)
set_Cycles_BNC(bnc_cer, 2, 2, send_errors=True, turnOffFirst=True)

configure_tc_windows(-1, input=4)
configure_tc_windows(0, input=3)

In [41]:
start_signals_per_sec = tc_check_start_signals()
print(start_signals_per_sec)

0


In [19]:
print(cyc_set_len)
print(len(pol_settings))
print(len(output_channels))
print(len(trials))
print(acquisition_num)
print(durs)
print(ave_acq_time)

4
34
1
2
272
[240, 330, 420, 270]
0:05:15


In [ ]:
# For rotating the motors a lot to test how reliable they are
x1 = 0 + 15;
while x1 <= 360:
    motor1.toPosition(x1)
    motor2.toPosition(x1)
    motor3.toPosition(x1)
    motor4.toPosition(x1)

    x1 += 15
    
    time.sleep(4)